<a href="https://colab.research.google.com/github/Thiiagoo777/Mielo1/blob/main/PIPELINE_FILTRAGEM_DE_RISCO_MF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **FILTRAGEM DE VARIANTES SOMÁTICAS - PAINEL MIELOFIBROSE PROGNÓSTICO RUIM**

**OBJETIVO**: Este *pipeline* se destina à filtrar variantes somáticas que ofereçam prognóstico ruim para a doença de mielofibrose.

- Foram filtradas 30 amostras do projeto "LMA Brasil".

- Os genes considerados para a filtragem foram:

- CALR, ASXL1, SRSF2, U2AF1 (baseado em artigo fundador do GIPSS – sistema de prognóstico por pontuação inspirado em dados genéticos https://doi.org/10.1038/s41375-018-0107-z)

## 1. Preparar ambiente de trabalho





In [ ]:
#Clona github do projeto lmabrasil-hg48.git
%%bash
rm -rf lmabrasil-hg38
git clone https://github.com/renatopuga/lmabrasil-hg38

Cloning into 'lmabrasil-hg38'...


In [ ]:
#Clona meu github contendo amostras do projeto lmabrasil convertidos para hg38
!git clone https://github.com/bioinfoamos01/projetolma.git

Cloning into 'projetolma'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 35 (delta 29), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (35/35), 28.22 MiB | 2.79 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [ ]:
#remove arquivo README.md da pasta lma
!rm /content/projetolma/README.md

In [ ]:
#Copia 30 arquivos da pasta projetolma(contendo amostras pós lift-over hg38) para a pasta /vep_output
!cp /content/projetolma/* /content/lmabrasil-hg38/vep_output

In [ ]:
#Cria uma lista.txt com 9 genes de risco para prognóstico ruim para mielofibrose
!echo -e "CALR\nASXL1\nSRSF2\nEZH2\nIDH1\nIDH2\nU2AF1\n" > /content/lmabrasil-hg38/hpo/mielofibrose.txt

## 2. Instalar bcftools com plugin (+split-vep)


In [ ]:
#permite extrair os campos de anotações estruturadas como INFO/CSQ criadas por VEP
%%bash
git clone --recurse-submodules https://github.com/samtools/htslib.git
git clone https://github.com/samtools/bcftools.git
cd bcftools
make
make install

Submodule path 'htscodecs': checked out 'ffda7310c4b3292955561d6c3b1743cb82bfe26b'
cd ../htslib && make htslib.pc.tmp
make[1]: Entering directory '/content/htslib'
echo '# Default htscodecs.mk generated by Makefile' > htscodecs.mk
echo 'include $(HTSPREFIX)htscodecs_bundled.mk' >> htscodecs.mk
./hts_probe_cc.sh 'gcc' '-g -Wall -O2 -fvisibility=hidden ' '-fvisibility=hidden' >> htscodecs.mk
sed -e '/^static_libs=/s/@static_LIBS@/-lz -lm -lbz2 -llzma -lcurl/;s#@[^-][^@]*@##g' htslib.pc.in > htslib.pc.tmp
make[1]: Leaving directory '/content/htslib'
cd ../htslib && make htslib_static.mk
make[1]: Entering directory '/content/htslib'
sed -n '/^static_libs=/s/[^=]*=/HTSLIB_static_LIBS = /p;/^static_ldflags=/s/[^=]*=/HTSLIB_static_LDFLAGS = /p' htslib.pc.tmp > htslib_static.mk
make[1]: Leaving directory '/content/htslib'
echo '/* Basic config.h generated by Makefile */' > config.h
echo '#define ENABLE_BCF_PLUGINS 1' >> config.h
echo '#define PLUGIN_EXT ".so"' >> config.h
echo '#define BCFTOOL

Cloning into 'htslib'...
Submodule 'htscodecs' (https://github.com/samtools/htscodecs.git) registered for path 'htscodecs'
Cloning into '/content/htslib/htscodecs'...
Cloning into 'bcftools'...


## 3. Instalar udocker


In [ ]:
%%bash
# Fonte: https://gist.github.com/mwufi/6718b30761cd109f9aff04c5144eb885
pip install udocker
udocker --allow-root install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 1.4 MB/s eta 0:00:00
Info: creating repo: /root/.udocker
Info: udocker command line interface 1.3.12
Info: searching for udockertools >= 1.2.10
Info: installing udockertools 1.2.10
Info: installation of udockertools successful


## 4. Baixar imagem do ensembl-vep


In [ ]:
!udocker --allow-root pull ensemblorg/ensembl-vep

Info: downloading layer sha256:d57d4eb4e8d71e1a3b27b0515d9ffb6f053291c0bf4667f22980dcfe84b6913a
Info: downloading layer sha256:8da5acaae4c91044ca467ac4bb74e41badcce7ed0e2e7f12e2d6c2a3f7333b6d
Info: downloading layer sha256:8a144600929dc8a2519022c13ef41d984920440d53af8b2736b928e56465eb16
Info: downloading layer sha256:e9c96579a6c7fc8199cd61ac60ea4fc2e083ef03e470ac5b9e0dabfd1f52adb8
Info: downloading layer sha256:63ae9cf8caa98c5e9c7695631faa795cd06a0debda97a55baa832625a52b0b74
Info: downloading layer sha256:50045e6409ebc1687df4548cfc330f1fad340dfa92b7a84ab3985da7642f5a71
Info: downloading layer sha256:a1ec7562e9cd6a332e524d590b8b20413457228487629b4a8f155a121f406ba2
Info: downloading layer sha256:a2230cfa06a7b5a218b0d0f23d53cd3af410839a6ed6ab9883502611ec9d321b
Info: downloading layer sha256:cebaa0fecec3b5b9a48b37aed9cc625da104156617c497e0440a03944ae7dc5f
Info: downloading layer sha256:4f4fb700ef54461cfa02571ae0db9a0dc1e0cdb5577484a6d75e68dc38e8acc1
Info: downloading layer sha256:9bec7112a

## 5. Filtragem

In [ ]:
#Declaração de lista com nome das 30 amostras do projeto lma brasil
SAMPLES = ["WP048","WP093","WP087","WP060","WP056","WP066","WP064","WP072","WP078","WP285","WP280","WP274","WP276","WP270","WP216","WP306","WP297","WP291","WP295","WP204","WP160","WP164","WP162","WP212","WP170","WP196","WP180","WP188","WP140","WP126"]

In [ ]:
#loop para processar pipeline de filtragem de variantes em cada uma das 30 amostras da lista SAMPLES
for i in SAMPLES:
  SAMPLE = i
  !echo {SAMPLE}
  !sh lmabrasil-hg38/vep-gc.sh {SAMPLE} mielofibrose.txt

WP048
 
 ****************************************************************************** 
 *                                                                            * 
 *               STARTING 3adea43c-b9dd-3d24-8f31-4ac88234a918                * 
 *                                                                            * 
 ****************************************************************************** 
 executing: filter_vep

-------------------- WARNING ----------------------
MSG: The script 'filter_vep' couldn't find the VEP annotations INFO field CSQ in 149 lines of the input file
FILE: src/ensembl-vep/filter_vep LINE: 345
CALLED BY: src/ensembl-vep/filter_vep  LINE: 55
Date (localtime)    = Sat Jan 27 22:10:43 2024
Ensembl API version = 111
---------------------------------------------------
WP093
 
 ****************************************************************************** 
 *                                                                            * 
 *              

In [ ]:

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

#converte as 30 amostras filtradas pelo vep-gc.sh em amostras .csv --> output salvo na pasta /content/lmabrasil-hg38/csv_filtrados
!mkdir /content/lmabrasil-hg38/csv_filtrados
for i in SAMPLES:
  df = pd.read_csv(f'/content/lmabrasil-hg38/vep_output/liftOver_{i}_hg19ToHg38.vep.filter.tsv',sep='\t',index_col=False)
  df.to_csv(f'/content/lmabrasil-hg38/csv_filtrados/{i}_filtrado.csv', index=False)

## **6. TABELA BRUTA COM VARIANTES DO PAINEL DE GENES (RESULTADO)**

Resultado da filtragem das 30 amostras para mutações de prognóstico ruim para mielofibrose - FALTANDO FILTRAGEM DE SCORE


In [ ]:
import glob
import pandas as pd

# Lista pasta com os 30 arquivos csv(s)
csv_files = glob.glob('/content/lmabrasil-hg38/csv_filtrados/*.{}'.format('csv'))

#Une o resultado dos 30 arquivos csv(s) numa única tabela
df_concat = pd.concat([pd.read_csv(i) for i in csv_files], ignore_index=True)

#OUTPUT TABELA FINAL EM CSV
!mkdir /content/lmabrasil-hg38/tabela_final
df_concat.to_csv('/content/lmabrasil-hg38/tabela_final/tabela_final.csv', index=False)

df_concat

,CHROM,POS,REF,ALT,Location,SYMBOL,Consequence,Feature,MANE_SELECT,BIOTYPE,HGVSc,HGVSp,EXON,INTRON,VARIANT_CLASS,SIFT,PolyPhen,gnomADg_AF,MAX_AF,IMPACT,CLIN_SIG,SOMATIC,Existing_variation,FILTER,TumorID,GT,DP,AD,AF,NormalID,NGT,NDP,NAD,NAF
0,chr19,12943750,AGCAGAGGCTTAAGGAGGAGGAAGAAGACAAGAAACGCAAAGAGGAGGAGGAG,A,chr19:12943751-12943802,CALR,frameshift_variant,NM_004343.4,ENST00000316448.10,protein_coding,NM_004343.4:c.1099_1150del,NP_004334.1:p.Leu367ThrfsTer46,9/9,.,deletion,.,.,0.000020,0.000066,HIGH,pathogenic,.,rs1555760738,map_qual,WP078,0/1,91,"59,32",0.406,WP079,0/0,46.0,"46,0",0.028
1,chr20,32434638,A,AG,chr20:32434638-32434639,ASXL1,frameshift_variant,NM_015338.6,ENST00000375687.10,protein_coding,NM_015338.6:c.1934dup,NP_056153.2:p.Gly646TrpfsTer12,13/13,.,insertion,.,.,0.000644,0.001739,HIGH,likely_pathogenic&pathogenic,0&1,rs750318549&COSV60102510,PASS,WP078,0/1,53,"25,28",0.536,WP079,0/0,31.0,"30,1",0.029
2,chr20,32434483,T,TA,chr20:32434483-32434484,ASXL1,stop_gained&frameshift_variant,NM_015338.6,ENST00000375687.10,protein_coding,NM_015338.6:c.1772dup,NP_056153.2:p.Tyr591Ter,13/13,.,insertion,.,.,0.000033,0.000094,HIGH,.,0&1&1,rs762036456&COSV60104310&COSV60124684,normal_artifact,WP093,0/1,312,"161,151",0.481,WP094,0/0,91.0,"86,5",0.073
3,chr21,43104346,G,A,chr21:43104346,U2AF1,missense_variant,NM_006758.3,ENST00000291552.9,protein_coding,NM_006758.3:c.101C>T,NP_006749.1:p.Ser34Phe,2/8,.,SNV,deleterious_low_confidence(0.01),probably_damaging(0.998),0.000000,0.000000,MODERATE,likely_pathogenic,0&1&1,rs371769427&COSV52341059&COSV52341472,PASS,WP216,0/1,106,"77,29",0.284,WP217,0/0,49.0,"48,1",0.044
4,chr20,32434789,C,T,chr20:32434789,ASXL1,stop_gained,NM_015338.6,ENST00000375687.10,protein_coding,NM_015338.6:c.2077C>T,NP_056153.2:p.Arg693Ter,13/13,.,SNV,.,.,0.000020,0.000062,HIGH,pathogenic&likely_pathogenic,0&1,rs373221034&COSV60102332,PASS,WP306,0/1,130,"77,53",0.434,WP307,0/0,132.0,"131,1",0.019
5,chr21,43094667,T,G,chr21:43094667,U2AF1,missense_variant,NM_006758.3,ENST00000291552.9,protein_coding,NM_006758.3:c.470A>C,NP_006749.1:p.Gln157Pro,6/8,.,SNV,deleterious_low_confidence(0),possibly_damaging(0.541),0.000073,0.000115,MODERATE,not_provided&likely_pathogenic,0&1&1,rs371246226&COSV52341120&COSV52341147,PASS,WP306,0/1,82,"49,33",0.429,WP307,0/0,60.0,"59,1",0.037
6,chr19,12943750,AGCAGAGGCTTAAGGAGGAGGAAGAAGACAAGAAACGCAAAGAGGAGGAGGAG,A,chr19:12943751-12943802,CALR,frameshift_variant,NM_004343.4,ENST00000316448.10,protein_coding,NM_004343.4:c.1099_1150del,NP_004334.1:p.Leu367ThrfsTer46,9/9,.,deletion,.,.,0.000020,0.000066,HIGH,pathogenic,.,rs1555760738,map_qual,WP297,0/1,66,"47,19",0.387,WP298,0/0,27.0,"27,0",0.045
7,chr19,12943813,A,ATTGTC,chr19:12943813-12943814,CALR,frameshift_variant,NM_004343.4,ENST00000316448.10,protein_coding,NM_004343.4:c.1154_1155insTTGTC,NP_004334.1:p.Lys385AsnfsTer47,9/9,.,insertion,.,.,0.000020,0.000094,HIGH,pathogenic,0&1&1,rs765476509&COSV57116551&COSV57118194,germline,WP276,0/1,30,"22,8",0.256,NaN,NaN,NaN,NaN,NaN
8,chr19,12943750,AGCAGAGGCTTAAGGAGGAGGAAGAAGACAAGAAACGCAAAGAGGAGGAGGAG,A,chr19:12943751-12943802,CALR,frameshift_variant,NM_004343.4,ENST00000316448.10,protein_coding,NM_004343.4:c.1099_1150del,NP_004334.1:p.Leu367ThrfsTer46,9/9,.,deletion,.,.,0.000020,0.000066,HIGH,pathogenic,.,rs1555760738,map_qual,WP188,0/1,61,"46,15",0.288,WP189,0/0,29.0,"29,0",0.041
9,chr19,12943750,AGCAGAGGCTTAAGGAGGAGGAAGAAGACAAGAAACGCAAAGAGGAGGAGGAG,A,chr19:12943751-12943802,CALR,frameshift_variant,NM_004343.4,ENST00000316448.10,protein_coding,NM_004343.4:c.1099_1150del,NP_004334.1:p.Leu367ThrfsTer46,9/9,.,deletion,.,.,0.000020,0.000066,HIGH,pathogenic,.,rs1555760738,map_qual;normal_artifact,WP160,0/1,45,"35,10",0.261,WP161,0/0,41.0,"40,1",0.059


## **7. TABELA COM SCORES DE RISCO - EM CRIAÇÃO**


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

df = pd.read_csv("/content/lmabrasil-hg38/tabela_final/tabela_final.csv")
#Extrai da tabela_final, em um csv resumido contendo informações das colunas "TumorID","HGVSc","SYMBOL", "HGVSp", "CHROM", "EXON"
df1 = df[["TumorID","SYMBOL", "HGVSc","HGVSp", "CHROM", "EXON"]]
df1.to_csv('/content/tabela_resumo_.csv', index=False)

#Converte a tabela resumida em uma lista
lista = df1.values.tolist()
S1 = []
variantes = []
var = []

#SEPARANDO O NOME DAS AMOSTRAS NÃO ENCONTRADAS NO FILTRO DE VARIANTES, NA VARIÁVEL S1, QUE COMPUTARÁ TODOS COMO +1 DEVIDO A AUSÊNCIA DE CALR TIPO1 PROTETOR)
for x in range(len(lista)):
  var = lista[x][0]
  variantes.append(var)

for i in SAMPLES:
  if i not in variantes:
    S1.append(i)

#dicionário para receber variantes, todos começam com pontuação +1, só são despontuados em -1 se for encontrado mutação tipo 1 em CALR
#PONTUA TODAS AS AMOSTRAS DA LISTA DE VARIANTES CONFORME CRITÉRIOS DA GISSP + GENES IDH1, IDH2 E EZH2
dicio = {}
for i in range(len(lista)):
  dicio[lista[i][0]] = 1


for i in range(len(lista)):
  if ("CALR" and "NM_004343.4:c.1099_1150del") in lista[i]: #mutação do tipo 1 em CARL
    for k, v in dicio.items():
      if lista[i][0] == k:
        dicio[lista[i][0]] +=-1

  if ("SRSF2" and "1/3") in lista[i]: #hostspot em exon 1 de SRSF2
    for k, v in dicio.items():
      if lista[i][0] == k:
        dicio[lista[i][0]] +=1

  if "ASXL1" in lista[i]: #hotspot cromossomo 12?
    for k, v in dicio.items():
      if lista[i][0] == k:
        dicio[lista[i][0]] +=1

  if ("U2AF1" and "NM_006758.3:c.470A>C") in lista[i]: #hotspot mutação Q157 EM U2AF1
    for k, v in dicio.items():
      if lista[i][0] == k:
        dicio[lista[i][0]] +=1

for i in range(len(S1)):
  dicio[S1[i]] = 1


#Computando as pontuações usando um dicionário para armazenar as contas

contagem = {}

for i in dicio.values():
  if i in contagem:
    contagem[i] +=1
  else:
    contagem[i] =1

#Exibindo os dados usando um dataframe pandas

import pandas as pd

data = {"TOTAL DE PACIENTES": [contagem[0],contagem[1],contagem[2],contagem[3]],
        "SCORE": ["0", "1", "2", ">=3"],
        "RISCO": ["Baixo", "Intermediário 1", "Intermediário 2", "Alto"],
        "SOBREVIDA EM 5 ANOS (%)": ["94,0%", "73,0%", "40,0%", "14,0%"],
        "SOBREVIDA MÉDIA (ANOS)": ["26,4 anos", "10,3 anos", "4,6 anos", "2,6 anos"]}

dataf = pd.DataFrame(data)

dataf.style.set_caption('ESTRATIFICADOR DE PROGNÓSTICO DE MIELOFIBROSE BASEADADO EM GIPSS (sem cariotipagem)')

,PACIENTES,SCORE,RISCO,SOBREVIDA EM 5 ANOS (%),SOBREVIDA MÉDIA (ANOS)
0,4,0,Baixo,"94,0%","26,4 anos"
1,21,1,Intermediário 1,"73,0%","10,3 anos"
2,4,2,Intermediário 2,"40,0%","4,6 anos"
3,1,>=3,Alto,"14,0%","2,6 anos"


In [ ]:

criterios = {"Penalidade": ["+1", "+1", "+1", "+1"],
        "Achados": ["Ausência de deleção de 52pb em CALR", "Mutação em Exon 1 de SRSF2", "Mutação em ASXL1", "Mutação Q157 em U2AF1"]}
criteriosf = pd.DataFrame(criterios)
criteriosf.style.set_caption('CRITÉRIOS DE PONTUAÇÃO BASEADO EM GIPSS (adaptado)')

,Penalidade,Achados
0,+1,Ausência de deleção de 52pb em CALR
1,+1,Mutação em Exon 1 de SRSF2
2,+1,Mutação em ASXL1
3,+1,Mutação Q157 em U2AF1
